In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

transform = make_column_transformer(
    (OneHotEncoder(), ["industry"]),
    remainder="passthrough"
)

model = LinearRegression(fit_intercept=False)
qt = QuantileTransformer(output_distribution="normal")
poly = PolynomialFeatures(degree=2, include_bias=False)

In [13]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

# Get data

In [15]:
df = pd.read_sql(
    """
    select date, ticker, bm, mom12m, roeq, siccd, mve, ret
    from data
    where date >= '2018-01'
    order by date, ticker
    """,
    conn
  )
df = df.dropna()
df = df.set_index(["date", "ticker"])
features = ["roeq", "bm", "mom12m"]

#### Drop largest 500 stocks each month

In [16]:
df["size_rnk"] = df.groupby("date").mve.rank(ascending=False)
df = df[df.size_rnk>500]

# Quantile Transform

In [17]:
grouped = df.groupby("date", group_keys=False)

df[features+["ret"]] = grouped[features+["ret"]].apply(
  lambda d: 
    pd.DataFrame(
      qt.fit_transform(d),
      columns=d.columns,
      index=d.index
    )     
)

## Add industry variable to dataframe


In [19]:
inds = pd.read_csv("siccodes12.csv", index_col="industry")
ind_names = inds.index.unique().to_list()

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"
    
codes = pd.Series({code: industry(code) for code in df.siccd.unique()})
codes.name = "industry" 
codes.index.name = "siccd"
df = df.merge(codes, on="siccd")
features.append("industry")

# Make Pipelines

In [21]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

transform1 = make_column_transformer(
    (OneHotEncoder(), ["industry"]),
    remainder="passthrough"
)
transform2 = PolynomialFeatures(degree=2)
pipe = make_pipeline(
    transform1,
    transform2,
    model
)

# Saving your pipeline

In [22]:
import joblib
joblib.dump(pipe, './my_pipe.joblib')

['./my_pipe.joblib']

# Loading your saved pipeline

In [23]:
pipe = joblib.load('./my_pipe.joblib')